In [480]:
%reload_ext autoreload
%autoreload 2
import chess.pgn
import numpy as np
from chess_dataset import ChessDataset, ChessPairDataset
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [481]:
chess_pair_dataset = ChessPairDataset("dataset/npy/")
train_chess_dataset = ChessDataset("dataset/npy/")
test_chess_dataset = ChessDataset("dataset/npy/", train=False)

In [482]:
len(chess_pair_dataset)

20000

In [641]:
chess_trainloader = DataLoader(train_chess_dataset, batch_size=64, shuffle=True)
chess_testloader = DataLoader(test_chess_dataset, batch_size=64, shuffle=False)
chess_pair_trainloader = DataLoader(chess_pair_dataset, batch_size=64, shuffle=True)

In [220]:
class Pos2Vec(nn.Module):
  def __init__(self):
    super(Pos2Vec, self).__init__()
    self.en1 = nn.Linear(773, 600)
    self.en2 = nn.Linear(600, 400)
    self.en3 = nn.Linear(400, 200)
    self.en4 = nn.Linear(200, 100)

    self.de1 = nn.Linear(600, 773)
    self.de2 = nn.Linear(400, 600)
    self.de3 = nn.Linear(200, 400)
    self.de4 = nn.Linear(100, 200)

    self.g1 = nn.Sequential(self.en1, self.de1, nn.Sigmoid())
    
    self.g2 = nn.Sequential(self.en1, nn.ReLU(), self.en2, nn.ReLU(),
                            self.de2, nn.ReLU(), self.de1, nn.Sigmoid())
    
    self.g3 = nn.Sequential(self.en1, nn.ReLU(), self.en2, nn.ReLU(), self.en3, nn.ReLU(),
                            self.de3, nn.ReLU(), self.de2, nn.ReLU(), self.de1, nn.Sigmoid())
    
    self.g4 = nn.Sequential(self.en1, nn.ReLU(), self.en2, nn.ReLU(), self.en3, nn.ReLU(), self.en4, nn.ReLU(),
                            self.de4, nn.ReLU(), self.de3, nn.ReLU(), self.de2, nn.ReLU(), self.de1, nn.Sigmoid())
    
    self.infer = nn.Sequential(self.en1, nn.ReLU(), self.en2, nn.ReLU(), self.en3, nn.ReLU(), self.en4, nn.ReLU())
    
  def forward(self, x):
    return self.curr(x)
  
  def train_all(self, train_data, val_data, epochs=10):
    print("773 - 600 -773")
    self.curr = self.g1
    self.train(train_data, val_data, epochs)

    # * means freeze the layer
    print("773 -* 600 - 400 - 600 -* 773")
    Pos2Vec.freeze(self.en1)
    Pos2Vec.freeze(self.de1)
    self.curr = self.g2
    self.train(train_data, val_data, epochs)
    
    print("773 -* 600 -* 400 - 200 - 400 -* 600 -* 773")
    Pos2Vec.freeze(self.en2)
    Pos2Vec.freeze(self.de2)
    self.curr = self.
    self.train(train_data, val_data, epochs)
    
    print("773 -* 600 -* 400 -* 200 - 100 - 200 -* 400 -* 600 -* 773")
    Pos2Vec.freeze(self.en3)
    Pos2Vec.freeze(self.de3)
    self.curr = self.g4
    self.train(train_data, val_data, epochs)
    
  def train(self, train_data, val_data, epochs=10):
    loss_f = nn.MSELoss()
    optim = torch.optim.Adam(self.parameters(), lr=0.0005)
    for epoch in range(epochs):
      optim.param_groups[0]['lr'] = 0.0005 * (0.98 ** epoch)
      for states in (t := tqdm(train_data)):
        reconstructed = self(states)
        optim.zero_grad()
        loss = loss_f(reconstructed, states)
        loss.backward()
        optim.step()

        t.set_description("Epoch: {} | Loss: {}".format(epoch, loss))
      # Validation run
      self.validation(val_data)

  def validation(self, val_data):
    loss_f = nn.MSELoss()
    for states in (t:= tqdm(val_data)):
      reconstructed = self(states)
      loss = loss_f(reconstructed, states)
      t.set_description("Val loss: {}".format(loss))

  def build(self):
    self.curr = self.g4
  
  def inference_mode(self):
    Pos2Vec.unfreeze(self.en1)
    Pos2Vec.unfreeze(self.en2)
    Pos2Vec.unfreeze(self.en3)
    self.curr = self.infer
    
  @staticmethod
  def freeze(layer):
    layer.weight.requires_grad = False
    layer.bias.requires_grad = False
  
  @staticmethod
  def unfreeze(layer):
    layer.weight.requires_grad = True
    layer.bias.requires_grad = True

In [221]:
p2v = Pos2Vec()

In [222]:
p2v.train_all(epochs=30, train_data=chess_trainloader, val_data=chess_testloader)
# p2v.build()
# p2v.train(train_data=chess_trainloader, val_data=chess_testloader, epochs=10)

773 - 600 -773


Epoch: 0 | Loss: 0.0301969051361084: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:01<00:00, 32.12it/s]
Val loss: 0.026164155453443527: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 39.37it/s]
Epoch: 1 | Loss: 0.024991821497678757: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:01<00:00, 33.42it/s]
Val loss: 0.022566121071577072: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 44.87it/s]
Epoch: 2 | Loss: 0.02127440832555294: 10

773 -* 600 - 400 - 600 -* 773


Epoch: 0 | Loss: 0.026506587862968445: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:01<00:00, 28.06it/s]
Val loss: 0.02453627996146679: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 40.16it/s]
Epoch: 1 | Loss: 0.021597355604171753: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:01<00:00, 28.24it/s]
Val loss: 0.02106691151857376: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 38.43it/s]
Epoch: 2 | Loss: 0.01898167096078396: 10

773 -* 600 -* 400 - 200 - 400 -* 600 -* 773


Epoch: 0 | Loss: 0.026959240436553955: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:02<00:00, 26.45it/s]
Val loss: 0.024152304977178574: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 40.01it/s]
Epoch: 1 | Loss: 0.019787542521953583: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:02<00:00, 26.42it/s]
Val loss: 0.019380295649170876: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 40.02it/s]
Epoch: 2 | Loss: 0.01708832010626793: 10

773 -* 600 -* 400 -* 200 - 100 - 200 -* 400 -* 600 -* 773


Epoch: 0 | Loss: 0.02055853046476841: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:02<00:00, 26.35it/s]
Val loss: 0.02160891331732273: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 46.77it/s]
Epoch: 1 | Loss: 0.017447588965296745: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:02<00:00, 24.95it/s]
Val loss: 0.01857905089855194: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 41.95it/s]
Epoch: 2 | Loss: 0.017522769048810005: 1

In [223]:
for state in chess_trainloader:
  for i in range(773):
    s = state[0][i]
    ret = p2v(state)[0][i]
    if s == 1:
      print(s, ret)
  break

tensor(1.) tensor(0.9939, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.9911, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.7213, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.9723, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.9976, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.9926, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.9300, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.6953, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.3781, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.9060, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.0046, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.9997, grad_fn=<SelectBackward0>)
tensor(1.) tensor(1.0000, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.9615, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.9975, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.8463, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.9883, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.9891, grad_fn=<SelectBackward0>)
tensor(1.) tensor(0.6059, grad_fn=<SelectBackw

In [224]:
torch.save(p2v.state_dict(), "model/save/pos2vec.pth")

In [644]:
class DeepChess(nn.Module):
  def __init__(self):
    super(DeepChess, self).__init__()
    self.pos2vec = Pos2Vec()
    # self.pos2vec.build()
    # self.pos2vec.load_state_dict(torch.load("model/save/pos2vec.pth"))
    self.pos2vec.inference_mode()
    
    self.dense1 = nn.Linear(200, 400)
    self.dense2 = nn.Linear(400, 200)
    self.dense3 = nn.Linear(200, 100)
    self.dense4 = nn.Linear(100, 2)
  
  def forward(self, input1, input2):
    first_board = self.pos2vec(input1)
    second_board = self.pos2vec(input2)
    concat = torch.cat((first_board, second_board), 1)
    retval = F.relu(self.dense1(concat))
    retval = F.relu(self.dense2(retval))
    retval = F.relu(self.dense3(retval))
    retval = F.softmax(self.dense4(retval),1)
    return retval

In [645]:
import helper

In [646]:
dp = DeepChess()

In [647]:
helper.train(dp, chess_pair_trainloader, 2)

Epoch: 0 | Loss: 0.0 | Accuracy: 1.0: 100%|███████████████████████████████████████████████████████████| 313/313 [00:16<00:00, 19.19it/s]
Epoch: 1 | Loss: 0.0 | Accuracy: 1.0: 100%|███████████████████████████████████████████████████████████| 313/313 [00:16<00:00, 19.12it/s]


In [648]:
i = next(iter(chess_pair_trainloader))
z = i[2]
zz = dp(i[0], i[1])
print(torch.argmax(z, 1))
#print(zz)
print(torch.argmax(zz, 1))
# for a in zip(z, zz):
#   print(a)

tensor([1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
        0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0])
tensor([1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
        0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0])


In [611]:
loss = nn.BCELoss()
nput = torch.tensor([[1.],[1.]])
target = torch.tensor([[1.],[0.]])
output = loss(nput, target)

In [612]:
output

tensor(50.)